### 2020707035 박시언 lab5
description as follow

1. None or Nan handling
2. do log1p transform
3. 학습 과정 모니터링을 위한 tqdm 사용
4. linear regression을 Wx + b & 편미분을 통한 학습
5. rand seed 고정이 되지 않아 epoch 고정 후 학습 시 성능이 제각각

In [645]:
from tkinter import YView, XView

import numpy as np
import pandas as pd
np.random.seed = 123
data = pd.read_csv("https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv")

In [646]:
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [647]:
data['Age'].fillna(data['Age'].mean(),inplace=True)
data['Cabin'].fillna('N',inplace=True)
data['Embarked'].fillna('N',inplace=True)

/var/folders/9d/lcchyqrx6jq_hz6pm4d1j7z80000gn/T/ipykernel_58162/3057721139.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Age'].fillna(data['Age'].mean(),inplace=True)
/var/folders/9d/lcchyqrx6jq_hz6pm4d1j7z80000gn/T/ipykernel_58162/3057721139.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values a

In [648]:
data.loc[data["Sex"] == "male", "Sex_encode"] = 0
data.loc[data["Sex"] == "female", "Sex_encode"] = 1

In [649]:
from sklearn.preprocessing import LabelEncoder

# Null 처리 함수
def fillna(df):
    df['Age'].fillna(df['Age'].mean(), inplace=True)
    df['Cabin'].fillna('N', inplace=True)
    df['Embarked'].fillna('N', inplace=True)
    df['Fare'].fillna(0, inplace=True)
    return df

# 머신러닝 알고리즘에 불필요한 피처 제거
def drop_features(df):
    df.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)
    return df

# 레이블 인코딩 수행.
def format_features(df):
    df['Cabin'] = df['Cabin'].str[:1]
    features = ['Cabin', 'Sex', 'Embarked']
    for feature in features:
        le = LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

# 앞에서 설정한 데이터 전처리 함수 호출
def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df

In [650]:
Y = data['Survived']
X = data.drop('Survived', axis=1)


In [651]:
X = transform_features(X)

/var/folders/9d/lcchyqrx6jq_hz6pm4d1j7z80000gn/T/ipykernel_58162/3807705002.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(df['Age'].mean(), inplace=True)
/var/folders/9d/lcchyqrx6jq_hz6pm4d1j7z80000gn/T/ipykernel_58162/3807705002.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alwa

In [652]:
import numpy as np

# 로그 변환 적용 (값이 0 이하인 경우를 방지하기 위해 작은 값 더하기)
X['Fare'] = np.log(X['Fare'] + 1)
X['Age'] = np.log(X['Age'] + 1)



In [653]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X, Y, \
                                                  test_size=0.2, random_state=11)

In [654]:
def batch_sampler(data, target, batch_size):
    tot = len(data)
    total_iter = tot // batch_size
    for i in range(total_iter):
        yield (data[i * batch_size:(i + 1) * batch_size],
               target[i * batch_size:(i + 1) * batch_size])


sampler = batch_sampler(data=X_train, target=y_train, batch_size=32)
L = len(X_train) // 32
for i in range(L):
    next(sampler)
print(X_train)

     Pclass  Sex       Age  SibSp  Parch      Fare  Cabin  Embarked  \
333       3    1  2.833213      2      0  2.944439      7         3   
662       1    1  3.871201      0      0  3.280441      4         3   
382       3    1  3.496508      0      0  2.188856      7         3   
331       1    1  3.839452      0      0  3.384390      2         3   
149       2    1  3.761200      0      0  2.639057      7         3   
..      ...  ...       ...    ...    ...       ...    ...       ...   
269       1    0  3.583519      0      0  4.917301      2         3   
337       1    0  3.737670      0      0  4.908972      4         0   
91        3    1  3.044522      0      0  2.180892      7         3   
80        3    1  3.135494      0      0  2.302585      7         3   
703       3    1  3.258097      0      0  2.168105      7         2   

     Sex_encode  
333         0.0  
662         0.0  
382         0.0  
331         0.0  
149         0.0  
..          ...  
269         1.0  
337

In [655]:
import numpy as np
from tqdm import tqdm
class LogisticRegression:
    def __init__(self, learning_rate=0.01, epochs=400,batch_size = 32, n_features=9):
        # 초기 가중치를 무작위로 설정
        self.weights = np.random.randn(n_features)
        self.b = np.random.randn()
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.loss_history = []
        self.batch_size = batch_size

    def sigmoid(self, z):
        """Sigmoid 활성화 함수"""
        return 1.0 / (1 + np.exp(-z))

    def compute_loss(self, preds, target):
        """교차 엔트로피 손실 계산"""
        # 로그의 안정성을 위해 값들을 클리핑
        preds = np.clip(preds, 1e-9, 1 - 1e-9)  # 0과 1의 범위를 벗어나지 않도록 함
        loss = -((target * np.log(preds)) + (1 - target) * np.log(1 - preds)).mean()
        return loss


    def fit(self, X, y, batch_sampler):
        """모델 훈련 함수"""
        for epoch in tqdm(range(self.epochs)):
            sampler = batch_sampler(data=X, target=y, batch_size=self.batch_size)
            batch_loss = []
    
            for X_batch, Y_batch in sampler:
                # 순방향 전파
                preds = self.sigmoid(np.dot(X_batch, self.weights) + self.b)
    
                # 손실 계산
                loss = self.compute_loss(preds, Y_batch)
                batch_loss.append(loss)
    
                # 그래디언트 계산
                dloss_dz = preds - Y_batch
                weights_deriv = np.dot(X_batch.T, dloss_dz) / X_batch.shape[0]
                b_deriv = dloss_dz.mean()
    
                # 가중치 및 바이어스 업데이트
                self.weights -= self.learning_rate * weights_deriv
                self.b -= self.learning_rate * b_deriv
    
            # Epoch별 손실 저장
            epoch_loss = np.mean(batch_loss)
            self.loss_history.append(epoch_loss)
            
            # 에포크마다 손실 출력
            print(f"Epoch {epoch + 1}/{self.epochs} - Loss: {epoch_loss:.4f}")


    def predict(self, X):
        """모델 예측 함수"""
        linear_comb = np.dot(X, self.weights) + self.b
        return self.sigmoid(linear_comb) > 0.55

    def accuracy(self, X, y):
        """정확도 계산 함수"""
        preds = self.predict(X)
        return (preds == y).mean() * 100




In [656]:
model = LogisticRegression(learning_rate=0.01, epochs=200,batch_size=32, n_features=9)
model.fit(X_train, y_train, batch_sampler=batch_sampler)
# X_test에 대한 예측값 생성
predictions = model.predict(X_test)

# 정확도 계산
def calculate_accuracy(predictions, y_test):
    """예측값과 실제값을 비교하여 정확도를 계산"""
    correct_predictions = (predictions == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100
    return accuracy

# y_test와 비교하여 정확도 계산
accuracy = calculate_accuracy(predictions, y_test)
print(f"Test Accuracy: {accuracy:.2f}%")


 16%|█▌        | 31/200 [00:00<00:01, 147.84it/s]

Epoch 1/200 - Loss: 7.0507
Epoch 2/200 - Loss: 1.9568
Epoch 3/200 - Loss: 0.9475
Epoch 4/200 - Loss: 0.9090
Epoch 5/200 - Loss: 0.8816
Epoch 6/200 - Loss: 0.8563
Epoch 7/200 - Loss: 0.8327
Epoch 8/200 - Loss: 0.8109
Epoch 9/200 - Loss: 0.7906
Epoch 10/200 - Loss: 0.7717
Epoch 11/200 - Loss: 0.7541
Epoch 12/200 - Loss: 0.7378
Epoch 13/200 - Loss: 0.7225
Epoch 14/200 - Loss: 0.7082
Epoch 15/200 - Loss: 0.6949
Epoch 16/200 - Loss: 0.6824
Epoch 17/200 - Loss: 0.6707
Epoch 18/200 - Loss: 0.6597
Epoch 19/200 - Loss: 0.6495
Epoch 20/200 - Loss: 0.6398
Epoch 21/200 - Loss: 0.6307
Epoch 22/200 - Loss: 0.6222
Epoch 23/200 - Loss: 0.6142
Epoch 24/200 - Loss: 0.6067
Epoch 25/200 - Loss: 0.5996
Epoch 26/200 - Loss: 0.5929
Epoch 27/200 - Loss: 0.5866
Epoch 28/200 - Loss: 0.5807
Epoch 29/200 - Loss: 0.5751
Epoch 30/200 - Loss: 0.5698
Epoch 31/200 - Loss: 0.5649


 30%|███       | 61/200 [00:00<00:00, 141.36it/s]

Epoch 32/200 - Loss: 0.5602
Epoch 33/200 - Loss: 0.5557
Epoch 34/200 - Loss: 0.5515
Epoch 35/200 - Loss: 0.5476
Epoch 36/200 - Loss: 0.5438
Epoch 37/200 - Loss: 0.5403
Epoch 38/200 - Loss: 0.5369
Epoch 39/200 - Loss: 0.5337
Epoch 40/200 - Loss: 0.5307
Epoch 41/200 - Loss: 0.5278
Epoch 42/200 - Loss: 0.5251
Epoch 43/200 - Loss: 0.5225
Epoch 44/200 - Loss: 0.5201
Epoch 45/200 - Loss: 0.5177
Epoch 46/200 - Loss: 0.5155
Epoch 47/200 - Loss: 0.5134
Epoch 48/200 - Loss: 0.5114
Epoch 49/200 - Loss: 0.5094
Epoch 50/200 - Loss: 0.5076
Epoch 51/200 - Loss: 0.5058
Epoch 52/200 - Loss: 0.5042
Epoch 53/200 - Loss: 0.5026
Epoch 54/200 - Loss: 0.5010
Epoch 55/200 - Loss: 0.4996
Epoch 56/200 - Loss: 0.4982
Epoch 57/200 - Loss: 0.4968
Epoch 58/200 - Loss: 0.4955
Epoch 59/200 - Loss: 0.4943
Epoch 60/200 - Loss: 0.4931
Epoch 61/200 - Loss: 0.4920
Epoch 62/200 - Loss: 0.4909
Epoch 63/200 - Loss: 0.4898
Epoch 64/200 - Loss: 0.4888
Epoch 65/200 - Loss: 0.4878
Epoch 66/200 - Loss: 0.4869
Epoch 67/200 - Loss:

 53%|█████▎    | 106/200 [00:00<00:00, 141.41it/s]

...

Epoch 107/200 - Loss: 0.4662
Epoch 108/200 - Loss: 0.4660
Epoch 109/200 - Loss: 0.4657
Epoch 110/200 - Loss: 0.4655
Epoch 111/200 - Loss: 0.4653
Epoch 112/200 - Loss: 0.4650
Epoch 113/200 - Loss: 0.4648
Epoch 114/200 - Loss: 0.4646
Epoch 115/200 - Loss: 0.4644
Epoch 116/200 - Loss: 0.4642
Epoch 117/200 - Loss: 0.4640
Epoch 118/200 - Loss: 0.4638
Epoch 119/200 - Loss: 0.4636
Epoch 120/200 - Loss: 0.4634
Epoch 121/200 - Loss: 0.4632
Epoch 122/200 - Loss: 0.4631
Epoch 123/200 - Loss: 0.4629
Epoch 124/200 - Loss: 0.4627
Epoch 125/200 - Loss: 0.4626
Epoch 126/200 - Loss: 0.4624
Epoch 127/200 - Loss: 0.4623
Epoch 128/200 - Loss: 0.4621
Epoch 129/200 - Loss: 0.4620
Epoch 130/200 - Loss: 0.4618
Epoch 131/200 - Loss: 0.4617
Epoch 132/200 - Loss: 0.4616
Epoch 133/200 - Loss: 0.4614
Epoch 134/200 - Loss: 0.4613
Epoch 135/200 - Loss: 0.4612
Epoch 136/200 - Loss: 0.4610
Epoch 137/200 - Loss: 0.4609
Epoch 138/200 - Loss: 0.4608
Epoch 139/200 - Loss: 0.4607
Epoch 140/200 - Loss: 0.4606
Epoch 141/200 

 90%|█████████ | 181/200 [00:01<00:00, 143.33it/s]

Epoch 152/200 - Loss: 0.4594
Epoch 153/200 - Loss: 0.4593
Epoch 154/200 - Loss: 0.4593
Epoch 155/200 - Loss: 0.4592
Epoch 156/200 - Loss: 0.4591
Epoch 157/200 - Loss: 0.4590
Epoch 158/200 - Loss: 0.4590
Epoch 159/200 - Loss: 0.4589
Epoch 160/200 - Loss: 0.4588
Epoch 161/200 - Loss: 0.4587
Epoch 162/200 - Loss: 0.4587
Epoch 163/200 - Loss: 0.4586
Epoch 164/200 - Loss: 0.4586
Epoch 165/200 - Loss: 0.4585
Epoch 166/200 - Loss: 0.4584
Epoch 167/200 - Loss: 0.4584
Epoch 168/200 - Loss: 0.4583
Epoch 169/200 - Loss: 0.4582
Epoch 170/200 - Loss: 0.4582
Epoch 171/200 - Loss: 0.4581
Epoch 172/200 - Loss: 0.4581
Epoch 173/200 - Loss: 0.4580
Epoch 174/200 - Loss: 0.4580
Epoch 175/200 - Loss: 0.4579
Epoch 176/200 - Loss: 0.4579
Epoch 177/200 - Loss: 0.4578
Epoch 178/200 - Loss: 0.4578
Epoch 179/200 - Loss: 0.4577
Epoch 180/200 - Loss: 0.4577
Epoch 181/200 - Loss: 0.4576
Epoch 182/200 - Loss: 0.4576
Epoch 183/200 - Loss: 0.4575
Epoch 184/200 - Loss: 0.4575
Epoch 185/200 - Loss: 0.4575
Epoch 186/200 

100%|██████████| 200/200 [00:01<00:00, 142.76it/s]

Epoch 197/200 - Loss: 0.4570
Epoch 198/200 - Loss: 0.4570
Epoch 199/200 - Loss: 0.4569
Epoch 200/200 - Loss: 0.4569
Test Accuracy: 87.15%


In [657]:
model = LogisticRegression(learning_rate=0.01, epochs=200,batch_size=64, n_features=9)
model.fit(X_train, y_train, batch_sampler=batch_sampler)
# X_test에 대한 예측값 생성
predictions = model.predict(X_test)

# 정확도 계산
def calculate_accuracy(predictions, y_test):
    """예측값과 실제값을 비교하여 정확도를 계산"""
    correct_predictions = (predictions == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100
    return accuracy

# y_test와 비교하여 정확도 계산
accuracy = calculate_accuracy(predictions, y_test)
print(f"Test Accuracy: {accuracy:.2f}%")


 14%|█▍        | 29/200 [00:00<00:00, 280.14it/s]

Epoch 1/200 - Loss: 2.6572
Epoch 2/200 - Loss: 1.8833
Epoch 3/200 - Loss: 1.4818
Epoch 4/200 - Loss: 1.3502
Epoch 5/200 - Loss: 1.2973
Epoch 6/200 - Loss: 1.2561
Epoch 7/200 - Loss: 1.2170
Epoch 8/200 - Loss: 1.1789
Epoch 9/200 - Loss: 1.1416
Epoch 10/200 - Loss: 1.1054
Epoch 11/200 - Loss: 1.0703
Epoch 12/200 - Loss: 1.0362
Epoch 13/200 - Loss: 1.0035
Epoch 14/200 - Loss: 0.9719
Epoch 15/200 - Loss: 0.9418
Epoch 16/200 - Loss: 0.9130
Epoch 17/200 - Loss: 0.8856
Epoch 18/200 - Loss: 0.8596
Epoch 19/200 - Loss: 0.8352
Epoch 20/200 - Loss: 0.8121
Epoch 21/200 - Loss: 0.7905
Epoch 22/200 - Loss: 0.7704
Epoch 23/200 - Loss: 0.7515
Epoch 24/200 - Loss: 0.7341
Epoch 25/200 - Loss: 0.7178
Epoch 26/200 - Loss: 0.7028
Epoch 27/200 - Loss: 0.6890
Epoch 28/200 - Loss: 0.6762
Epoch 29/200 - Loss: 0.6644
Epoch 30/200 - Loss: 0.6536
Epoch 31/200 - Loss: 0.6436
Epoch 32/200 - Loss: 0.6344
Epoch 33/200 - Loss: 0.6260
Epoch 34/200 - Loss: 0.6182
Epoch 35/200 - Loss: 0.6110
Epoch 36/200 - Loss: 0.6044
E

 29%|██▉       | 58/200 [00:00<00:00, 280.42it/s]

Epoch 59/200 - Loss: 0.5334
Epoch 60/200 - Loss: 0.5320
Epoch 61/200 - Loss: 0.5306
Epoch 62/200 - Loss: 0.5293
Epoch 63/200 - Loss: 0.5280
Epoch 64/200 - Loss: 0.5268
Epoch 65/200 - Loss: 0.5257
Epoch 66/200 - Loss: 0.5246
Epoch 67/200 - Loss: 0.5235
Epoch 68/200 - Loss: 0.5224
Epoch 69/200 - Loss: 0.5214
Epoch 70/200 - Loss: 0.5204
Epoch 71/200 - Loss: 0.5195
Epoch 72/200 - Loss: 0.5186
Epoch 73/200 - Loss: 0.5177
Epoch 74/200 - Loss: 0.5168
Epoch 75/200 - Loss: 0.5160
Epoch 76/200 - Loss: 0.5151
Epoch 77/200 - Loss: 0.5143
Epoch 78/200 - Loss: 0.5136
Epoch 79/200 - Loss: 0.5128
Epoch 80/200 - Loss: 0.5121
Epoch 81/200 - Loss: 0.5113
Epoch 82/200 - Loss: 0.5106
Epoch 83/200 - Loss: 0.5099
Epoch 84/200 - Loss: 0.5093
Epoch 85/200 - Loss: 0.5086
Epoch 86/200 - Loss: 0.5079
Epoch 87/200 - Loss: 0.5073


 58%|█████▊    | 116/200 [00:00<00:00, 281.79it/s]

...

Epoch 117/200 - Loss: 0.4926
Epoch 118/200 - Loss: 0.4922
Epoch 119/200 - Loss: 0.4918
Epoch 120/200 - Loss: 0.4915
Epoch 121/200 - Loss: 0.4911
Epoch 122/200 - Loss: 0.4907
Epoch 123/200 - Loss: 0.4904
Epoch 124/200 - Loss: 0.4900
Epoch 125/200 - Loss: 0.4897
Epoch 126/200 - Loss: 0.4894
Epoch 127/200 - Loss: 0.4890
Epoch 128/200 - Loss: 0.4887
Epoch 129/200 - Loss: 0.4884
Epoch 130/200 - Loss: 0.4880
Epoch 131/200 - Loss: 0.4877
Epoch 132/200 - Loss: 0.4874
Epoch 133/200 - Loss: 0.4871
Epoch 134/200 - Loss: 0.4868
Epoch 135/200 - Loss: 0.4865
Epoch 136/200 - Loss: 0.4862
Epoch 137/200 - Loss: 0.4859
Epoch 138/200 - Loss: 0.4856
Epoch 139/200 - Loss: 0.4853
Epoch 140/200 - Loss: 0.4850
Epoch 141/200 - Loss: 0.4848
Epoch 142/200 - Loss: 0.4845
Epoch 143/200 - Loss: 0.4842
Epoch 144/200 - Loss: 0.4839
Epoch 145/200 - Loss: 0.4837
Epoch 146/200 - Loss: 0.4834
Epoch 147/200 - Loss: 0.4832
Epoch 148/200 - Loss: 0.4829
Epoch 149/200 - Loss: 0.4827
Epoch 150/200 - Loss: 0.4824
Epoch 151/200 

 87%|████████▋ | 174/200 [00:00<00:00, 282.91it/s]

Epoch 155/200 - Loss: 0.4812
Epoch 156/200 - Loss: 0.4810
Epoch 157/200 - Loss: 0.4807
Epoch 158/200 - Loss: 0.4805
Epoch 159/200 - Loss: 0.4803
Epoch 160/200 - Loss: 0.4801
Epoch 161/200 - Loss: 0.4799
Epoch 162/200 - Loss: 0.4796
Epoch 163/200 - Loss: 0.4794
Epoch 164/200 - Loss: 0.4792
Epoch 165/200 - Loss: 0.4790
Epoch 166/200 - Loss: 0.4788
Epoch 167/200 - Loss: 0.4786
Epoch 168/200 - Loss: 0.4784
Epoch 169/200 - Loss: 0.4782
Epoch 170/200 - Loss: 0.4780
Epoch 171/200 - Loss: 0.4778
Epoch 172/200 - Loss: 0.4776
Epoch 173/200 - Loss: 0.4774
Epoch 174/200 - Loss: 0.4773
Epoch 175/200 - Loss: 0.4771
Epoch 176/200 - Loss: 0.4769
Epoch 177/200 - Loss: 0.4767
Epoch 178/200 - Loss: 0.4765
Epoch 179/200 - Loss: 0.4764
Epoch 180/200 - Loss: 0.4762
Epoch 181/200 - Loss: 0.4760
Epoch 182/200 - Loss: 0.4758
Epoch 183/200 - Loss: 0.4757
Epoch 184/200 - Loss: 0.4755
Epoch 185/200 - Loss: 0.4753
Epoch 186/200 - Loss: 0.4752
Epoch 187/200 - Loss: 0.4750
Epoch 188/200 - Loss: 0.4749
Epoch 189/200 

100%|██████████| 200/200 [00:00<00:00, 281.46it/s]

Test Accuracy: 84.92%
